In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

models = []
model_preds_train = np.zeros(shape=(num_models, 188533))
model_preds_test = np.zeros(shape=(num_models, 125690))

for model_number in range(num_models):
    A = np.random.randn(X_data_num.shape[1], X_data_num.shape[1])
    Q, R = np.linalg.qr(A)
    X_data_num_transformed = X_data_num # @ Q
    X_data = pd.concat([pd.DataFrame(X_data_num_transformed), X_data_cat.reset_index(drop=True)], axis=1)
    
    fold_size = 188533 // fold_number
    indices = np.arange(188533)
    np.random.shuffle(indices)
    
    train_predictions = np.zeros(188533)
    test_predictions = np.zeros(125690)

    print(f"Now training model number {model_number}")

    for fold_idx in range(fold_number):
        val_indices = indices[fold_idx * fold_size:(fold_idx + 1) * fold_size]
        train_indices = np.concatenate([indices[:fold_idx * fold_size], indices[(fold_idx + 1) * fold_size:]])

        X_train_fold, y_train_fold = X_data.iloc[train_indices], y_train.iloc[train_indices]
        X_val_fold, y_val_fold = X_data.iloc[val_indices], y_train.iloc[val_indices]

        train_data = lgb.Dataset(X_train_fold, label=y_train_fold, categorical_feature=categorical_features,)
        valid_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data, categorical_feature=categorical_features)
        model = lgb.train(
            params,
            train_data,
            num_boost_round=params['n_estimators'],
            valid_sets=[valid_data],
            callbacks=[lgb.early_stopping(stopping_rounds=50)]
        )

        train_predictions[val_indices] = model.predict(X_val_fold, num_iteration=model.best_iteration)
        test_predictions += model.predict(X_data.iloc[188533:].reset_index(drop=True), num_iteration=model.best_iteration) * (1 / fold_number)
    
    train_predictions[0] = y_train.iloc[0]

    rmse = mean_squared_error(train_predictions, y_train)**(0.5)
    
    model_preds_train[model_number] = train_predictions
    model_preds_test[model_number] = test_predictions